# MLFlow Deployment with Explanations


Learn how to implement scoring-time explanations for [MLflow](https://www.mlflow.org/) model. This tutorial produces (1) a new model whose `predict()` method returns both predictions and explanations and (2) a custom deployment of that model to an [online endpoint](https://docs.microsoft.com/azure/machine-learning/concept-endpoints) for that model.

The "explanations" for this model come in the form of local feature importance values. Local feature importance measures the contribution of features for a specific prediction. This tutorial leverages Microsoft's [Responsible AI Toolbox](https://github.com/microsoft/responsible-ai-toolbox) to generate these values, which uses a Mimic explainer, also known as a global surrogate model. You can learn more in the Interpret ML Book's [chapter on global surrogates](https://christophm.github.io/interpretable-ml-book/global.html).


![Example of deployment with explanations](assets/DeploymentDiabetesExample.png "Example of deployment with explanations")


#### Requirements


- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Learn about workspaces](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace)


Running this notebook relies on packages in the `requirements.txt` file. Install them or run the command below:


In [ ]:
%pip install -r requirements.txt

We also need to give the name of the compute cluster we want to use in AzureML. Later in this notebook, we will create it if it does not already exist:


In [ ]:
compute_name = "rai-cluster"

Finally, we need to specify a version for the data and components we will create while running this notebook. This should be unique for the workspace, but the specific value doesn't matter:


In [ ]:
diabetes_regression_example_version_string = "7"

## Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the MLClient from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace.


In [ ]:
# Enter details of your AML workspace
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [ ]:
# Handle to the workspace
import mlflow
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential


ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)
print(f"ML Client: {ml_client}")

azureml_tracking_uri = ml_client.workspaces.get(
    ml_client.workspace_name
).mlflow_tracking_uri
mlflow.set_tracking_uri(azureml_tracking_uri)
print(f"Tracking URI: {azureml_tracking_uri}")

In [ ]:
# Get handle to azureml registry for the RAI built in components
registry_name = "azureml"
ml_client_registry = MLClient(
    credential=DefaultAzureCredential(),
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    registry_name=registry_name,
)
print(ml_client_registry)

## Accessing the data

First, we need to obtain the dataset and upload it to our AzureML workspace:


In [ ]:
train_data_path = "data-diabetes-regression/train/"

In [ ]:
test_data_path = "data-diabetes-regression/test/"

Load some data for a quick view:


In [ ]:
import os
import pandas as pd
import mltable

tbl = mltable.load(train_data_path)
train_df: pd.DataFrame = tbl.to_pandas_dataframe()

# test dataset should have less than 5000 rows
test_df = mltable.load(test_data_path).to_pandas_dataframe()
assert len(test_df.index) <= 5000

display(train_df)

We are going to create two Datasets in AzureML, one for the train and one for the test datasets.


In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

input_train_data = "diabetes_regression_train_mltable"
input_test_data = "diabetes_regression_test_mltable"

try:
    # Try getting data already registered in workspace
    train_data = ml_client.data.get(
        name=input_train_data, version=diabetes_regression_example_version_string
    )
    test_data = ml_client.data.get(
        name=input_test_data, version=diabetes_regression_example_version_string
    )
except Exception as e:
    train_data = Data(
        path=train_data_path,
        type=AssetTypes.MLTABLE,
        description="Diabetes regression example training data",
        name=input_train_data,
        version=diabetes_regression_example_version_string,
    )
    ml_client.data.create_or_update(train_data)

    test_data = Data(
        path=test_data_path,
        type=AssetTypes.MLTABLE,
        description="Diabetes regression example test data",
        name=input_test_data,
        version=diabetes_regression_example_version_string,
    )
    ml_client.data.create_or_update(test_data)

## A model training pipeline

To simplify the model creation process, we're going to use a pipeline. This will have two stages:

1. The actual training component
1. A model registration component

We have to register the model in AzureML in order for our RAI insights components to use it.

### The Training Component

The training component is for this particular model. In this case, we are going to train a `RandomForestRegressor` on the input data and save it using MLFlow. We need command line arguments to specify the location of the input data, the location where MLFlow should write the output model, and the name of the target column in the dataset.

We start by creating a directory to hold the component source:


In [ ]:
import os

os.makedirs("component_src", exist_ok=True)
os.makedirs("register_model_src", exist_ok=True)

Next, put our training script into the directory:


In [ ]:
%%writefile component_src/diabetes_regression_training_script.py

import argparse
import os
import shutil
import tempfile

import mlflow
import mlflow.sklearn

import mltable

import pandas as pd
from sklearn.ensemble import RandomForestRegressor

def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--training_data", type=str, help="Path to training data")
    parser.add_argument("--target_column_name", type=str, help="Name of target column")
    parser.add_argument("--model_output", type=str, help="Path of output model")

    # parse args
    args = parser.parse_args()

    # return args
    return args


def main(args):
    tracking_uri = mlflow.get_tracking_uri()
    print("tracking_uri: {0}".format(tracking_uri))

    # Read in data
    print("Reading data")
    tbl = mltable.load(args.training_data)
    all_data = tbl.to_pandas_dataframe()

    print("Extracting X_train, y_train")
    print("all_data cols: {0}".format(all_data.columns))
    y_train = all_data[args.target_column_name]
    X_train = all_data.drop(labels=args.target_column_name, axis="columns")
    print("X_train cols: {0}".format(X_train.columns))

    print("Training model")
    # The estimator can be changed to suit
    model = RandomForestRegressor()
    model.fit(X_train, y_train)

    # Saving model with mlflow - leave this section unchanged
    with tempfile.TemporaryDirectory() as td:
        print("Saving model with MLFlow to temporary directory")
        tmp_output_dir = os.path.join(td, "my_model_dir")
        mlflow.sklearn.save_model(sk_model=model, path=tmp_output_dir)

        print("Copying MLFlow model to output path")
        for file_name in os.listdir(tmp_output_dir):
            print("  Copying: ", file_name)
            # As of Python 3.8, copytree will acquire dirs_exist_ok as
            # an option, removing the need for listdir
            shutil.copy2(src=os.path.join(tmp_output_dir, file_name), dst=os.path.join(args.model_output, file_name))


# run script
if __name__ == "__main__":
    # add space in logs
    print("*" * 60)
    print("\n\n")

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")

In [ ]:
%%writefile register_model_src/register.py

# ---------------------------------------------------------
# Copyright (c) Microsoft Corporation. All rights reserved.
# ---------------------------------------------------------

import argparse
import json
import os
import time
import tempfile

import mlflow
import mlflow.sklearn

# Based on example:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-train-cli
# which references
# https://github.com/Azure/azureml-examples/tree/main/cli/jobs/train/lightgbm/iris


def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--model_input_path", type=str, help="Path to input model")
    parser.add_argument(
        "--model_info_output_path", type=str, help="Path to write model info JSON"
    )
    parser.add_argument(
        "--model_base_name", type=str, help="Name of the registered model"
    )
    parser.add_argument(
        "--model_name_suffix", type=int, help="Set negative to use epoch_secs"
    )

    # parse args
    args = parser.parse_args()

    # return args
    return args


def main(args):
    tracking_uri = mlflow.get_tracking_uri()
    print("tracking_uri: {0}".format(tracking_uri))

    print("Loading model")
    mlflow_model = mlflow.sklearn.load_model(args.model_input_path)

    if args.model_name_suffix < 0:
        suffix = int(time.time())
    else:
        suffix = args.model_name_suffix
    registered_name = "{0}_{1}".format(args.model_base_name, suffix)
    print(f"Registering model as {registered_name}")

    # This is currently broken with latest azureml-mlflow package
    # print("Registering via MLFlow")
    # mlflow.sklearn.log_model(
    #     sk_model=mlflow_model,
    #     registered_model_name=registered_name,
    #     artifact_path=registered_name,
    # )
    print("Logging model via MLFlow using save_model approach")
    # Use save_model and create_model_version with file:// URI to let Azure ML handle the upload
    with tempfile.TemporaryDirectory() as temp_dir:
        model_dir = os.path.join(temp_dir, registered_name)
        mlflow.sklearn.save_model(mlflow_model, model_dir)

        # Use the older model registry API directly to avoid logged-models search
        from mlflow.tracking import MlflowClient
        client = MlflowClient()

        try:
            # Try to create the registered model (will fail if it already exists)
            client.create_registered_model(registered_name)
            print(f"Created new registered model: {registered_name}")
        except Exception as e:
            print(f"Registered model {registered_name} already exists: {e}")

        # Create a new version of the model using file:// URI
        # Azure ML will handle the upload and generate the proper azureml:// URI
        file_uri = f"file://{model_dir}"
        print("Registering model with file_uri: {0}".format(file_uri))

        model_version = client.create_model_version(
            name=registered_name,
            source=file_uri
        )
        print(f"Created model version {model_version.version} for {registered_name}")

    print("Writing JSON")
    dict = {"id": "{0}:1".format(registered_name)}
    output_path = os.path.join(args.model_info_output_path, "model_info.json")
    with open(output_path, "w") as of:
        json.dump(dict, fp=of)


# run script
if __name__ == "__main__":
    # add space in logs
    print("*" * 60)
    print("\n\n")

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")


Now that the training script is saved on our local drive, we create a YAML file to describe it as a component to AzureML. This involves defining the inputs and outputs, specifing the AzureML environment which can run the script, and telling AzureML how to invoke the training script:


In [ ]:
from azure.ai.ml import load_component

yaml_contents = (
    f"""
$schema: http://azureml/sdk-2-0/CommandComponent.json
name: rai_diabetes_regression_training_component
display_name: Diabetes regression training component for RAI example
version: {diabetes_regression_example_version_string}
type: command
inputs:
  training_data:
    type: path
  target_column_name:
    type: string
outputs:
  model_output:
    type: path
code: ./component_src/
environment: azureml://registries/azureml/environments/responsibleai-tabular/versions/28
"""
    + r"""
command: >-
  python diabetes_regression_training_script.py
  --training_data ${{{{inputs.training_data}}}}
  --target_column_name ${{{{inputs.target_column_name}}}}
  --model_output ${{{{outputs.model_output}}}}
"""
)

yaml_filename = "DiabetesRegressionTrainingComponent.yaml"

with open(yaml_filename, "w") as f:
    f.write(yaml_contents.format(yaml_contents))

train_model_component = load_component(source=yaml_filename)

In [ ]:
yaml_contents = f"""
$schema: http://azureml/sdk-2-0/CommandComponent.json
name: register_model
display_name: Register Model
version: {diabetes_regression_example_version_string}
type: command
is_deterministic: False
inputs:
  model_input_path:
    type: path
  model_base_name:
    type: string
  model_name_suffix: # Set negative to use epoch_secs
    type: integer
    default: -1
outputs:
  model_info_output_path:
    type: path
code: ./register_model_src/
environment: azureml://registries/azureml/environments/responsibleai-tabular/versions/28
command: >-
  python register.py
  --model_input_path ${{{{inputs.model_input_path}}}}
  --model_base_name ${{{{inputs.model_base_name}}}}
  --model_name_suffix ${{{{inputs.model_name_suffix}}}}
  --model_info_output_path ${{{{outputs.model_info_output_path}}}}

"""

yaml_filename = "register.yaml"

with open(yaml_filename, "w") as f:
    f.write(yaml_contents)

register_component = load_component(source=yaml_filename)

### Running a training pipeline

The component to register the model is part of the suite of RAI components, so we do not have to define it here. As such, we are now ready to run the training pipeline itself.

We start by ensuring that the compute cluster named above exists:

In [ ]:
from azure.ai.ml.entities import AmlCompute

all_compute_names = [x.name for x in ml_client.compute.list()]

if compute_name in all_compute_names:
    print(f"Found existing compute: {compute_name}")
else:
    my_compute = AmlCompute(
        name=compute_name,
        size="Standard_D2_v2",
        min_instances=0,
        max_instances=4,
        idle_time_before_scale_down=3600,
    )
    ml_client.compute.begin_create_or_update(my_compute).result()
    print("Initiated compute creation")

We continue by defining the name under which we want to register the model:

In [ ]:
import time

model_name_suffix = int(time.time())
model_name = "diabetes_decision_model"

Next, we define the pipeline using objects from the AzureML SDKv2. As mentioned above, there are two component jobs: one to train the model, and one to register it:

In [ ]:
from azure.ai.ml import dsl, Input

target_column = "y"

diabetes_train_pq = Input(
    type="mltable",
    path=f"azureml:{input_train_data}:{diabetes_regression_example_version_string}",
    mode="download",
)
diabetes_test_pq = Input(
    type="mltable",
    path=f"azureml:{input_test_data}:{diabetes_regression_example_version_string}",
    mode="download",
)


@dsl.pipeline(
    compute=compute_name,
    description="Register Model for Diabetes Decision Making example",
    experiment_name=f"Diabetes_Decision_Example_Model_Training_{model_name_suffix}",
)
def my_training_pipeline(target_column_name, training_data):
    trained_model = train_model_component(
        target_column_name=target_column_name, training_data=training_data
    )
    trained_model.set_limits(timeout=3600)

    _ = register_component(
        model_input_path=trained_model.outputs.model_output,
        model_base_name=model_name,
        model_name_suffix=model_name_suffix,
    )

    return {}


model_registration_pipeline_job = my_training_pipeline(target_column, diabetes_train_pq)

With the pipeline definition created, we can submit it to AzureML. We define a helper function to do the submission, which waits for the submitted job to complete:

In [ ]:
from azure.ai.ml.entities import PipelineJob
from IPython.core.display import HTML
from IPython.display import display


def submit_and_wait(ml_client, pipeline_job) -> PipelineJob:
    created_job = ml_client.jobs.create_or_update(pipeline_job)
    assert created_job is not None

    print("Pipeline job can be accessed in the following URL:")
    display(HTML('<a href="{0}">{0}</a>'.format(created_job.studio_url)))

    while created_job.status not in [
        "Completed",
        "Failed",
        "Canceled",
        "NotResponding",
    ]:
        time.sleep(30)
        created_job = ml_client.jobs.get(created_job.name)
        print("Latest status : {0}".format(created_job.status))
    assert created_job.status == "Completed"
    return created_job


# This is the actual submission
training_job = submit_and_wait(ml_client, model_registration_pipeline_job)

In [ ]:
model_id = f"{model_name}_{model_name_suffix}"
azureml_model_id = f"azureml:{model_id}:1"

## Create Wrapper Model


This tutorial uses a wrapper model to provide explanations along with predictions of the original model. The code for this wrapper can be reviewed in `explanation_wrapper.py`.


### Model information **(user input required)**


The `wrapper_model_name` will be the name of your model with explanations.


In [ ]:
# Name for the model with explanations
wrapper_model_name = f"diabetes_model_with_explanations_{model_name_suffix}"

In [ ]:
full_model_name = f"{model_name}_{model_name_suffix}"
model_uri = f"models:/{full_model_name}/1"

### Load baseline data


The data used to create the explanation wrapper must match the data used to train the original model. Be sure to drop any columns that were dropped during training.


In [ ]:
import pandas as pd
import mltable


data_asset = ml_client.data.get(
    name=input_train_data, version=diabetes_regression_example_version_string
)
baseline_df = mltable.load(train_data_path).to_pandas_dataframe()
# Drop any columns that were dropped when training the model. Uncomment and fill in first parameter
# baseline_df = baseline_df.drop([], axis="columns")

print("Baseline Data (first 5 rows):")
baseline_df.head()

### Create instance of ExplanationWrapper


In [ ]:
from explanation_wrapper import ExplanationWrapper

task_type = "regression"
categorical_features = []
explanation_wrapper = ExplanationWrapper(
    model_uri=model_uri,
    baseline_df=baseline_df,
    target_column=target_column,
    task_type=task_type,
    categorical_features=categorical_features,
)

## Register Wrapper Model


### Download original model **(user input required)**


Download the original model.


In [ ]:
# Create local folder
import os

local_path = "./artifacts/original_model"
if not os.path.exists(local_path):
    os.makedirs(local_path)

In [ ]:
import mlflow
from mlflow.tracking.client import MlflowClient

# Initialize MLFlow client
mlflow_client = MlflowClient()

In [ ]:
# Download run's artifacts/outputs using the latest version of the model
mlflow.artifacts.download_artifacts(artifact_uri=model_uri, dst_path=local_path)
print("Artifacts downloaded in: {}".format(local_path))
print("Artifacts: {}".format(os.listdir(local_path)))

In [ ]:
mlflow_model_dir = os.path.join(local_path, full_model_name)
model_file_path = f"./artifacts/original_model/{full_model_name}"
# Show the contents of the MLFlow model folder
os.listdir(mlflow_model_dir)

# You should see a list of files such as the following:
# ['artifacts', 'conda.yaml', 'MLmodel', 'python_env.yaml', 'python_model.pkl', 'requirements.txt']

### Get original model signature


If there is one, load the model signature from Azure ML.


In [ ]:
from mlflow.models import Model

mlflow_model = Model.load(model_uri)
model_signature = mlflow_model.signature
print(f"Model Signature: {model_signature}")

### Register wrapper model


Save and then register the model to your workspace. If you are re-running this step, you may need to delete the folder containing the last version of the saved wrapper model.


In [ ]:
artifacts = {"model": model_file_path, "RAI insights": "./artifacts/RAI_Insights"}

In [ ]:
mlflow.pyfunc.save_model(
    path=f"artifacts/{wrapper_model_name}",
    code_paths=["./explanation_wrapper.py"],
    conda_env="./env.yml",
    python_model=explanation_wrapper,
    artifacts=artifacts,
    signature=model_signature,
)

In [ ]:
mlflow.register_model(f"file://artifacts/{wrapper_model_name}", wrapper_model_name)

## 4. Create the scoring Endpoint


### Endpoint Details

In [ ]:
import hashlib
import re


def _make_endpoint_name(prefix: str, unique_source: str, max_len: int = 32) -> str:
    safe_prefix = re.sub(r"[^a-z0-9-]", "-", prefix.lower()).strip("-")
    suffix = hashlib.sha1(unique_source.encode("utf-8")).hexdigest()[:8]
    name = f"{safe_prefix}-{suffix}"
    name = re.sub(r"-{2,}", "-", name).strip("-")
    if not name[0].isalpha():
        name = f"e-{name}"
    return name[:max_len].rstrip("-")


# Stable, workspace-specific name to avoid regional name collisions in CI
endpoint_name = _make_endpoint_name(
    prefix="diabetes-decision-endpoint",
    unique_source=f"{subscription_id}|{resource_group}|{workspace}",
)

print(f"Endpoint name: {endpoint_name}")

In [ ]:
from azure.core.exceptions import HttpResponseError
from mlflow.deployments import get_deploy_client

deployment_client = get_deploy_client(azureml_tracking_uri)

try:
    deployment_client.create_endpoint(endpoint_name)
    print(f"Created endpoint: {endpoint_name}")
except HttpResponseError as e:
    # Endpoint names are unique per entire region; if the endpoint already exists, reuse it.
    if "already exists" in str(e).lower():
        print(f"Endpoint already exists; reusing: {endpoint_name}")
    else:
        raise

 ## Deploy Wrapper Model

### Get wrapper model ID


Verify that the model name and version are correct for the newly registered wrapper model.


In [ ]:
version_list = list(ml_client.models.list(wrapper_model_name))
wrapper_model_version = version_list[0].version
wrapper_model = ml_client.models.get(wrapper_model_name, wrapper_model_version)
print(f"Using model name: {wrapper_model_name}, version: {wrapper_model_version}")

#### Create and deploy endpoint


This step may take a while.


In [ ]:
from azure.ai.ml.entities import (
    OnlineRequestSettings,
    ManagedOnlineDeployment,
    ProbeSettings,
)

# Define the deployment
deployment_name = "mlflow-deploy-with-explanations"
deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=endpoint_name,
    model=wrapper_model.id,
    instance_count=1,
    request_settings=OnlineRequestSettings(request_timeout_ms=90000),
    liveness_probe=ProbeSettings(
        failure_threshold=30,
        success_threshold=1,
        period=100,
        initial_delay=500,
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=30,
        success_threshold=1,
        period=100,
        initial_delay=500,
    ),
)

# Trigger the deployment creation
try:
    ml_client.begin_create_or_update(deployment).wait()
    print("\n---Deployment created successfully---\n")
except Exception as err:
    raise RuntimeError(
        f"Deployment creation failed. Detailed Response:\n{err}"
    ) from err

###  Assign all traffic to the deployment


Create the traffic configuration:


In [ ]:
import json

traffic_config = {"traffic": {deployment_name: 100}}
traffic_config_path = "artifacts/traffic_config.json"
with open(traffic_config_path, "w") as outfile:
    outfile.write(json.dumps(traffic_config))

Update the configuration:


In [ ]:
from mlflow.deployments import get_deploy_client

deployment_client = get_deploy_client(mlflow.get_tracking_uri())
deployment_client.update_endpoint(
    endpoint=endpoint_name,
    config={"endpoint-config-file": traffic_config_path},
)

## Test Deployment


Testing the deployment can be done through the following steps or the UI in Azure ML portal.


Enter the number of samples you plan to use to test the endpoint. We will keep it simple in this example and just use one.


In [ ]:
n = 1

#### Option 1: Import data from csv

In [ ]:
# here we're dropping the target column, which in this diabetes dataset is called "target" rather than "y" although it represents the same data
sample = (
    pd.read_csv("https://azuremlexamples.blob.core.windows.net/datasets/diabetes.csv")
    .sample(n)
    .drop(columns=["target"])
    .reset_index(drop=True)
)

#### Option 2: Write data


Write data directly into the dataframe.


In [ ]:
data = {
    "age": [0.0380759064334241],
    "sex": [0.0506801187398187],
    "bmi": [0.0616962065186885],
    "bp": [0.0218723549949558],
    "s1": [-0.0442234984244464],
    "s2": [-0.0348207628376986],
    "s3": [-0.0434008456520269],
    "s4": [-0.00259226199818282],
    "s5": [0.0199084208763183],
    "s6": [-0.0176461251598052],
}

In [ ]:
if n == 1:
    sample = pd.DataFrame(data=data, index=[0])
else:
    sample = pd.DataFrame(data=data)
print(f"Sample Data: {sample.head()}")

### Invoke the endpoint


Get payload of `predict()` from the endpoint:


In [ ]:
payload = deployment_client.predict(endpoint=endpoint_name, df=sample)

Extract predictions and explanations from payload:


In [ ]:
import numpy as np

if isinstance(payload, pd.DataFrame):
    print("Return type is DataFrame")
    predictions = payload["predictions"].values
    explanations = payload["explanations"].values
elif isinstance(payload, np.ndarray):
    print("Return type is ndarray")
    predictions = payload.item()["predictions"]
    explanations = payload.item()["explanations"]
else:
    print(
        "Return type not supported - either skip the rest of this notebook or write your own code to extract the predictions and explanations lists"
    )

Print and view:


In [ ]:
features = np.array(baseline_df.drop(columns=[target_column]).columns)
if task_type == "classification":
    classes = np.array(baseline_df[target_column].unique())

for i in range(len(predictions)):
    print(f"For data point {i}:")
    print(f"{sample.loc[i]}\n")

    print(f"Prediction: {predictions[i]}\n")

    if task_type == "classification":
        for j in range(len(classes)):
            importances = np.array(explanations[i][j][0])
            explanations_df = pd.DataFrame(
                data={"feature": features, "local importance": importances}
            )
            print(f"Feature importances for class: {classes[j]}")
            if predictions[i] == classes[j]:
                print("This is the predicted class for this row of data")
            else:
                print("This is NOT the predicted class for this row of data")
            print(f"{explanations_df}\n")
    else:
        importances = np.array(explanations[i][0])
        explanations_df = pd.DataFrame(
            data={"feature": features, "local importance": importances}
        )
        print(f"Feature importances: {explanations_df}\n")

    print("\n\n\n")

## Explore data (Optional)


### Import plotly


In [ ]:
import plotly.express as px

### Feature importances for regression


In [ ]:
if task_type == "regression":
    for i in range(len(explanations)):
        explanations[i] = explanations[i][0]

    features = np.array(baseline_df.drop(columns=[target_column]).columns)
    for i in range(len(predictions)):
        print(f"For data point {i}:")
        print(f"{sample.loc[i]}\n")

        importances = np.array(explanations[i])
        explanations_df = pd.DataFrame(
            data={"feature": features, "local importance": importances}
        )

        plot = px.bar(
            data_frame=explanations_df,
            x="feature",
            y="local importance",
            title="Local Feature Importance",
        )
        plot.show()

        print("\n\n\n")
else:
    print("Task type is not regression - skip this section")

### Feature importances for classification


In [ ]:
if task_type == "classification":
    features = np.array(baseline_df.drop(columns=[target_column]).columns)
    classes = np.array(baseline_df[target_column].unique())
    for i in range(len(predictions)):
        print(f"For data point {i}:")
        print(f"{sample.loc[i]}\n")

        aggregated_df = pd.DataFrame(columns=["class", "feature", "local importance"])
        for j in range(len(classes)):
            importances = explanations[i][j][0]
            explanations_df = pd.DataFrame(
                data={"feature": features, "local importance": importances}
            )

            for k in range(len(features)):
                new_row = pd.DataFrame(
                    data=[[classes[j], features[k], explanations[i][j][0][k]]],
                    columns=["class", "feature", "local importance"],
                )
                aggregated_df = pd.concat([aggregated_df, new_row])

            title = f"Local Importance for Class {classes[j]}"
            if predictions[i] == classes[j]:
                title += " (Predicted Class)"
            else:
                title += " (Not the Predicted Class)"
            plot = px.bar(
                data_frame=explanations_df,
                x="feature",
                y="local importance",
                title=title,
            )
            plot.show()

        plot = px.bar(
            data_frame=aggregated_df,
            x="feature",
            y="local importance",
            color="class",
            title="Aggregated Local Importance",
        )
        plot.show()

        print("\n\n\n")
else:
    print("Task type is not classification - skip this section")